In [1]:
import numpy as np
from scipy.stats import truncnorm
from scipy.special import expit as activation_function


In [2]:
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [12]:
class NeuralNetwork:
  def __init__(self,no_of_in_nodes,no_of_out_nodes,no_of_hidden_nodes,learning_rate,bias=None):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.learning_rate = learning_rate
    self.bias = bias
    self.create_weight_matrices()

  def create_weight_matrices(self):
    bias_node = 1 if self.bias else 0
    rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes + bias_node))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_hidden_out = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes
    + bias_node))

  def train(self, input_vector, target_vector):
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size,1)
    if self.bias:
      input_vector = np.concatenate( (input_vector, [[self.bias]]) )
      target_vector = np.array(target_vector).reshape(target_vector.size, 1)
      output_vector_hidden = activation_function(self.weights_in_hidden @ input_vector)
      if self.bias:
        output_vector_hidden = np.concatenate( (output_vector_hidden, [[self.bias]]) )
      output_vector_network = activation_function(self.weights_hidden_out @ output_vector_hidden)
      output_error = target_vector - output_vector_network
      tmp = output_error * output_vector_network * (1.0 - output_vector_network)
      self.weights_hidden_out += self.learning_rate * (tmp @ output_vector_hidden.T)
      hidden_errors = self.weights_hidden_out.T @ output_error
      tmp = hidden_errors * output_vector_hidden * (1.0 - output_vector_hidden)
      if self.bias:
        x = (tmp @input_vector.T)[:-1,:]
      else:
        x = tmp @ input_vector.T
        self.weights_in_hidden += self.learning_rate * x

  def run(self, input_vector):
   input_vector = np.array(input_vector)
   input_vector = input_vector.reshape(input_vector.size, 1)
   if self.bias:
    input_vector = np.concatenate( (input_vector, [[1]]) )
    input4hidden = activation_function(self.weights_in_hidden@ input_vector)
    if self.bias:
      input4hidden = np.concatenate( (input4hidden, [[1]]) )
    output_vector_network = activation_function(self.weights_hidden_out @ input4hidden)
    return output_vector_network

  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i].argmax():
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs

  def print_weights(self):
    print("Weights from input to hidden layer:")
    print(self.weights_in_hidden)
    print("Weights from hidden to output layer:")
    print(self.weights_hidden_out)




In [13]:
# Create an instance of NeuralNetwork
nn = NeuralNetwork(no_of_in_nodes=2, no_of_out_nodes=1, no_of_hidden_nodes=3, learning_rate=0.1, bias=1)

# Train your neural network (assuming you have training data)
# nn.train(input_vector, target_vector)

# Print the weights
nn.print_weights()

Weights from input to hidden layer:
[[-0.44603458 -0.44251721 -0.50386364]
 [-0.14864624  0.01711142 -0.27582963]
 [ 0.49770027  0.53320251  0.34050988]]
Weights from hidden to output layer:
[[-0.12362029  0.39937206 -0.29248496 -0.08437417]]
